# Neural Net vs. Goldman
Can an LTSM Neural Net trained on fundamentals extracted from edgar XBRL from the S&P500 pick the same long and short list as Goldman's hedgefund meta list (appologies to my high school english teacher for the run on sentence)

## Data Sources
Download price and fundamental data for the S&P 500 using [pystock-crawler](https://github.com/eliangcs/pystock-crawler):  

`pystock-crawler reports ../tickers.csv -o ../reports.csv ../reports.log`  
`pystock-crawler prices ../tickers.csv -o ../prices.csv -l ../prices.log`

REMIND: Use pystock-crawler symbols to get symbols for training input...

X = 4 quarters of fundamental data and whether stock was up or down from the prior quarter y = whether the stock was up or down x days after the period that X was comprised of

In [1]:
import pandas
prices = pandas.read_csv('prices.csv', parse_dates=['date'], index_col=1)
reports = pandas.read_csv('reports.csv', parse_dates=['date'], index_col=1)
symbols = pandas.read_csv('symbols.csv')

In [2]:
# Add closing price and delta from next period i.e. did it go up 3 months from now
# instead make it more binary ie 1 is went up by X% using group df1 = df[df['budget'] > 10000]
XY = pandas.DataFrame()
for s in symbols['symbol']:
    try:
        r = reports[reports.symbol == s].sort(ascending=True)
        p = prices[prices.symbol == s].sort(ascending=True)
        r['close'] = r.index.map(lambda x: p.ix[p.index[p.index.searchsorted(x)]]['close'])
        r['delta'] = (r['close'].shift(-1) - r['close']).fillna(0)
        XY = XY.append(r)
    except:
        print "Problems parsing", s
print "Found", len(XY['symbol'].unique()), "symbols"

Problems parsing DISCK
Found 474 symbols


,symbol,amend,period_focus,fiscal_year,doc_type,revenues,op_income,net_income,eps_basic,eps_diluted,...,assets,cur_assets,cur_liab,cash,equity,cash_flow_op,cash_flow_inv,cash_flow_fin,close,delta
2009-06-30,MMM,False,Q2,2009,10-Q,5719000000,1191000000,783000000,1.12,1.12,...,26095000000,9898000000,5318000000,2596000000,11221000000,2161000000,-375000000,-1127000000,60.10,13.70
2009-09-30,MMM,False,Q3,2009,10-Q,6193000000,1478000000,957000000,1.36,1.35,...,27633000000,11196000000,5766000000,3239000000,12919000000,3897000000,-1076000000,-1417000000,73.80,8.87
2009-12-31,MMM,False,FY,2009,10-K,23123000000,4814000000,3193000000,4.56,4.52,...,27250000000,10795000000,4897000000,3040000000,13302000000,4941000000,-1732000000,-2014000000,82.67,0.90
2010-03-31,MMM,False,Q1,2010,10-Q,6348000000,1445000000,930000000,1.31,1.29,...,28022000000,12106000000,5148000000,2848000000,13851000000,1082000000,-987000000,-275000000,83.57,-4.58
2010-06-30,MMM,False,Q2,2010,10-Q,6731000000,1596000000,1121000000,1.57,1.54,...,28496000000,12891000000,5437000000,3022000000,14263000000,2220000000,-1438000000,-698000000,78.99,7.72


In [4]:
# Split into train/test sets
# Should use sklearn.cross_validation.StratifiedShuffleSplit to try and maintain industry sector % in each
# Or bin by financial size http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/
# s = XY['symbol'].unique()
from sklearn.cross_validation import train_test_split
unique_symbols = pandas.unique(XY['symbol'].values.ravel())
train_symbols, test_symbols = train_test_split(unique_symbols, test_size = 0.2)
print "num train symbols:", len(train_symbols)
train = XY[XY['symbol'].isin(train_symbols)]
test = XY[XY['symbol'].isin(test_symbols)]
print "test/train", 1.0 * len(test)/len(train)

num train symbols: 379
test/train 0.253043685844


In [5]:
# Convert into pure numerical matrices for training by dropping non-numeric columsn and split into X/y
X_train = train.ix[:,5:-1].values
y_train = train['delta'].values
X_test = test.ix[:,5:-1].values
y_test = test['delta'].values

In [7]:
import numpy
with open("train_test.npz", "wb") as f:
    numpy.savez(f, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)